In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, GlobalAveragePooling2D
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img


print("Constructing model")
model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(224, 224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))G
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, (3, 3), activation='relu',name='last_feature_layer'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
# you can download the pretrained weights from the following link
# https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing
# or you can find the detailed documentation https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

from keras.models import model_from_json
print("loading weights")
model.load_weights('vgg_face_weights.h5')
print("weights loaded")

In [14]:
# pad_model = Model(inputs=model.layers[0].input, outputs=model.get_layer('last_feature_layer').output)
x = GlobalAveragePooling2D()(model.get_layer('last_feature_layer').output)
x = Dense(100,activation='relu')(x)
x = Dense(1,activation='tanh')(x)
pad_model = Model(inputs=model.layers[0].input, outputs=x)


for layer in pad_model.layers:
    layer.trainable = False
pad_model.layers[-1].trainable = True
pad_model.layers[-2].trainable = True

pad_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_27_input (Inp (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_27 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_28 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 114, 114, 64)      0         
__________

In [3]:
# model.pop()
# model.add(Dense(100, activation='relu'))
# model.add(Dense(1, activation='tanh'))
# 
# #set only last layer to be trainable
# for layer in model.layers:
#     layer.trainable = False
# model.layers[-1].trainable = True
# model.layers[-2].trainable = True
# model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
__________

In [ ]:
# model.summary()
# vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output) # Functional model
# vgg_face_descriptor.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1_input (Inpu (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 114, 114, 64)      0         
__________

In [ ]:
import os

def getTrainingData():
    train_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs/attack_hand"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/attack_hand/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/ReplayAttackIMGs/real"):
        train_imgs.append(preprocess_image("datasets/ReplayAttackIMGs/real/" + file))
        labels.append(1)
    train_imgs = np.array(train_imgs)
    train_imgs = train_imgs[:, 0, :, :]
    labels = np.array(labels)
    return train_imgs, labels

tr_imgs, tr_labels = getTrainingData()
print(tr_imgs.shape, tr_labels.shape)


def getTestData():
    test_imgs = []
    labels = []
    for file in os.listdir("datasets/ReplayAttackIMGs_test/attack_hand"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/attack_hand/" + file))
        labels.append(-1)
    for file in os.listdir("datasets/ReplayAttackIMGs_test/real"):
        test_imgs.append(preprocess_image("datasets/ReplayAttackIMGs_test/real/" + file))
        labels.append(1)
    test_imgs = np.array(test_imgs)
    test_imgs = test_imgs[:, 0, :, :]
    labels = np.array(labels)
    return test_imgs, labels

test_imgs, test_labels = getTestData()
print(test_imgs.shape, test_labels.shape)


In [18]:
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy

# model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])
pad_model.compile(optimizer='adam', loss='mse', metrics=['accuracy',binary_accuracy])


In [22]:
history = pad_model.fit(tr_imgs, tr_labels, batch_size=16, epochs=1)
print(history.history)
pad_model.save("padModel.h5")

Epoch 1/1


  16/2400 [..............................] - ETA: 9:44 - loss: 1.5000 - acc: 0.6250 - binary_accuracy: 0.6250

  32/2400 [..............................] - ETA: 9:14 - loss: 1.5000 - acc: 0.6250 - binary_accuracy: 0.6250

  48/2400 [..............................] - ETA: 9:02 - loss: 1.4167 - acc: 0.6458 - binary_accuracy: 0.6458

  64/2400 [..............................] - ETA: 8:55 - loss: 1.3125 - acc: 0.6719 - binary_accuracy: 0.6719

...

2352/2400 [============================>.] - ETA: 11s - loss: 1.5000 - acc: 0.6250 - binary_accuracy: 0.6250

2368/2400 [============================>.] - ETA: 7s - loss: 1.4983 - acc: 0.6254 - binary_accuracy: 0.6254 

2384/2400 [============================>.] - ETA: 3s - loss: 1.4950 - acc: 0.6263 - binary_accuracy: 0.6263

2400/2400 [==============================] - 582s 243ms/step - loss: 1.5000 - acc: 0.6250 - binary_accuracy: 0.6250


{'acc': [0.625], 'binary_accuracy': [0.625], 'loss': [1.5]}


In [ ]:
# history = model.fit(tr_imgs, tr_labels, batch_size=1, epochs=2)
# print(history.history)
# model.save("MyVGGface.h5")

In [24]:
from keras.models import load_model
# model = load_model("MyVGGface.h5")
pad_model = load_model("padModel.h5")

In [26]:
score = pad_model.evaluate(test_imgs,test_labels,batch_size=16 )
print(score)

  16/3200 [..............................] - ETA: 12:56

  32/3200 [..............................] - ETA: 12:13

  48/3200 [..............................] - ETA: 12:05

  64/3200 [..............................] - ETA: 11:57

  80/3200 [..............................] - ETA: 11:47

...

3152/3200 [============================>.] - ETA: 12s

3168/3200 [============================>.] - ETA: 8s 

3184/3200 [============================>.] - ETA: 4s

3200/3200 [==============================] - 850s 266ms/step


[1.5, 0.625, 0.625]


In [ ]:
scores = pad_model.predict(test_imgs,batch_size=16,verbose=1)